In [31]:
import random
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

In [32]:
# Ejemplo de condición (puedes personalizarla según tus necesidades)
def keep(matriz):
    num_paradas = 34
    if np.sum(matriz) != num_paradas:
        return False
    elif np.trace(np.linalg.matriz_power(matriz, num_paradas)) <=1:
        return False
    elif np.all(np.diag(matriz) != 0):
        return False
    elif np.all(np.sum(matriz, axis=1) > 1):
        return False
    elif np.all(np.sum(matriz, axis=0) > 1):
        return False
    else:
        return True

In [33]:
def generar_grafo_aleatorio(num_nodos, num=200000):
    lista_grafos = []
    lista_matrices = []
    for _ in range(num):
        # Crear una matriz de adyacencia vacía (de tamaño num_nodos x num_nodos)
        matriz_adyacencia = np.zeros((num_nodos, num_nodos))

        # Aplicar condiciones sobre la matriz de adyacencia
        for i in range(num_nodos):
            for j in range(num_nodos):
                # Probabilidad de generar una arista de i a j
                matriz_adyacencia[i, j] = random.choice([0, 1])
        if keep(matriz_adyacencia):
            lista_matrices.append(matriz_adyacencia)
            # Crear un grafo dirigido usando la matriz de adyacencia
            G = nx.DiGraph()

            for i in range(num_nodos):
                for j in range(num_nodos):
                    if matriz_adyacencia[i, j] != 0:
                        G.add_edge(i, j)
            
            lista_grafos.append(G)

    return lista_grafos, lista_matrices


In [34]:
# Parámetros de entrada
num_nodos = 100


# Generar el grafo aleatorio y su matriz de adyacencia
lista_grafos, lista_matrices = generar_grafo_aleatorio(num_nodos)

# Mostrar la matriz de adyacencia
for G, matriz_adyacencia in zip(lista_grafos, lista_matrices):
    print("Matriz de adyacencia generada:")
    print(matriz_adyacencia)

    # Dibujar el grafo generado
    plt.figure(figsize=(8, 6))
    nx.draw(G, with_labels=True, node_size=500, node_color='lightblue', font_size=10, font_weight='bold', arrowsize=20)
    plt.title('Grafo Dirigido Aleatorio')
    plt.show()
